In [1]:
import os
import re
import imp
import binascii
from tempfile import NamedTemporaryFile

from IPython.core.magic import Magics, magics_class, cell_magic, line_magic
from IPython.display import display, HTML

@magics_class
class ProtoMagics(Magics):
    @cell_magic
    def schema(self, line, cell):
        module_name = line
        temp_dir = os.path.abspath('tmp_proto')
        fname = '{}/{}.proto'.format(temp_dir, module_name)
        cmd = "protoc -I={temp_dir} --python_out={temp_dir} {fname}"
        with open(fname, 'w') as f:
            f.write('syntax = "proto3";\n')
            f.write('package {};\n'.format(module_name))
            f.write(cell)
            f.write('\n')
        qualified_cmd = cmd.format(temp_dir=temp_dir, fname=fname)
        res = self.shell.getoutput(qualified_cmd)
        if res:
            print(res)
            raise Exception
        compiled_fname = '{}/{}_pb2.py'.format(temp_dir, module_name)
        globals()[module_name] = imp.load_source(module_name, compiled_fname)
            
    @cell_magic
    def represent(self, line, cell):
        serialized = eval(cell).SerializeToString().decode('latin-1')
        if line == 'braille':
            encodechar = (
                u"⠀⢀⠠⢠⠐⢐⠰⢰⠈⢈⠨⢨⠘⢘⠸⢸" +
                u"⡀⣀⡠⣠⡐⣐⡰⣰⡈⣈⡨⣨⡘⣘⡸⣸" +
                u"⠄⢄⠤⢤⠔⢔⠴⢴⠌⢌⠬⢬⠜⢜⠼⢼" +
                u"⡄⣄⡤⣤⡔⣔⡴⣴⡌⣌⡬⣬⡜⣜⡼⣼" +
                u"⠂⢂⠢⢢⠒⢒⠲⢲⠊⢊⠪⢪⠚⢚⠺⢺" +
                u"⡂⣂⡢⣢⡒⣒⡲⣲⡊⣊⡪⣪⡚⣚⡺⣺" +
                u"⠆⢆⠦⢦⠖⢖⠶⢶⠎⢎⠮⢮⠞⢞⠾⢾" +
                u"⡆⣆⡦⣦⡖⣖⡶⣶⡎⣎⡮⣮⡞⣞⡾⣾" +
                u"⠁⢁⠡⢡⠑⢑⠱⢱⠉⢉⠩⢩⠙⢙⠹⢹" +
                u"⡁⣁⡡⣡⡑⣑⡱⣱⡉⣉⡩⣩⡙⣙⡹⣹" +
                u"⠅⢅⠥⢥⠕⢕⠵⢵⠍⢍⠭⢭⠝⢝⠽⢽" +
                u"⡅⣅⡥⣥⡕⣕⡵⣵⡍⣍⡭⣭⡝⣝⡽⣽" +
                u"⠃⢃⠣⢣⠓⢓⠳⢳⠋⢋⠫⢫⠛⢛⠻⢻" +
                u"⡃⣃⡣⣣⡓⣓⡳⣳⡋⣋⡫⣫⡛⣛⡻⣻" +
                u"⠇⢇⠧⢧⠗⢗⠷⢷⠏⢏⠯⢯⠟⢟⠿⢿" +
                u"⡇⣇⡧⣧⡗⣗⡷⣷⡏⣏⡯⣯⡟⣟⡿⣿"
            )
            braille = u''.join(encodechar[ord(x)] for x in serialized)
            print(braille)
        elif line == 'binary':
            binary = u' '.join(u'{:0>8b}'.format(ord(x)) for x in serialized)
            print(binary)
        elif line == 'hex':
            v = [binascii.hexlify(x.encode('latin-1')).decode('latin-1') for x in serialized]
            hexrepr = u' '.join(v)
            print(hexrepr)
        elif line == 'dec':
            dec = u' '.join('{}'.format(ord(x)) for x in serialized)
            print(dec)
        else:
            print(' '.join(repr(x)[1:-1] for x in serialized))
        print("Length: {}".format(len(serialized)))
        
get_ipython().register_magics(ProtoMagics)

# Protocol buffers from the wire, up

## Why not JSON?
![json is bad](https://pbs.twimg.com/media/DN8NEOlX0AAhpV1.jpg:large)

## Why not JSON?
![json is really bad](https://pbs.twimg.com/media/DN8NEGlX0AABTwe.jpg:large)

## Why not JSON?

In [22]:
len(re.sub(r'\s+', '', '''

{"value": 123, 
 "produced": {
   "time": "11:30", 
   "offset": 0
 },
 "people": [
   {"first": "Alice", "last": "Smith", "status": "ACTIVE"},
   {"first": "Bob", "last": "Johnson", "status": "INACTIVE"}
 ],
 }
 
'''))

105

## Protobuf

In [3]:
%%schema foo
message Produced {
    string time = 1;
    int32 offset = 2;
}
message Person {
    enum Status {
        ACTIVE = 0;
        INACTIVE = 1;
    }
    string first = 1;
    string last = 2;
    Status status = 3;
}
message Payload {
    uint32 value = 1;
    Produced produced = 2;
    repeated Person people = 3;
}

## Protobuf

In [27]:
%%represent 
foo.Payload(
    value=123,
    produced=foo.Produced(time="11:30", offset=0),
    people=[
        foo.Person(first="Alice", last="Smith", status=foo.Person.ACTIVE),
        foo.Person(first="Bob", last="Johnson", status=foo.Person.INACTIVE)
    ]
)

\x08 { \x12 \x07 \n \x05 1 1 : 3 0 \x1a \x0e \n \x05 A l i c e \x12 \x05 S m i t h \x1a \x10 \n \x03 B o b \x12 \x07 J o h n s o n \x18 \x01
Length: 45


## Protobuf

In [28]:
%%represent binary
foo.Payload(value=123)

00001000 01111011
Length: 2


In [6]:
%%represent binary
foo.Person(
  first="a",
  last="bc",
  status=foo.Person.INACTIVE
)

00001010 00000001 01100001 00010010 00000010 01100010 01100011 00011000 00000001
Length: 9


In [7]:
%%represent 
foo.Payload(
  people=[foo.Person(last='x'), foo.Person(last='y')]
)

\x1a \x03 \x12 \x01 x \x1a \x03 \x12 \x01 y
Length: 10


## Protobuf pitfalls

In [8]:
foo.Person.FromString(b'').status is foo.Person.ACTIVE

True

## Protobuf pitfalls

In [9]:
%%schema foo2
message Produced {
    string time = 1;
    int32 offset = 2;
}
message Person {
    enum Status {
        ACTIVE = 0;
        INACTIVE = 1;
    }
    string first = 1;
    string last = 2;
    Status status = 3;
}
message Payload {
    uint32 value = 1;
    Produced produced = 2;
    repeated Person people = 3;
}

## Protobuf pitfalls

In [10]:
%%schema bar1
message Bar {
  int32 val = 1;
}

In [11]:
%%schema bar2
message Bar {
  string val = 1;
}

In [12]:
bar2.Bar.FromString(
  bar1.Bar(val=32).SerializeToString()
)

In [13]:
_.val

''

In [26]:
%%represent binary
bar2.Bar(val="32")

00001010 00000010 00110011 00110010
Length: 4


## Protobuf pitfalls

In [15]:
%%schema sizes1
message Shirt {
  enum Size {
    SMALL = 0;
    LARGE = 1;
  }
  Size size = 1;
}

In [16]:
%%schema sizes2
message Shirt {
  enum Size {
    SMALL = 0;
    LARGE = 1;
    MEDIUM = 2;
  }
  Size size = 1;
}

In [17]:
sizes1.Shirt.FromString(
  sizes2.Shirt(size=sizes2.Shirt.MEDIUM).SerializeToString()
).size

2

In [18]:
sizes1.Shirt.Size.Name(_)

ValueError: Enum Size has no name defined for value 2